In [1]:
# create workspace
# azureml-core of version 1.0.72 or higher is required
# azureml-dataprep[pandas] of version 1.1.34 or higher is required
from azureml.core import Workspace, Dataset, Datastore, ScriptRunConfig
from azureml.data.data_reference import DataReference
import os


In [1]:
# create a pipeline
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import Pipeline
from azureml.data.datapath import DataPath, DataPathComputeBinding
from azureml.pipeline.core import PipelineParameter
from azureml.core.datastore import Datastore
from azureml.pipeline.core import PipelineData
from azureml.data import OutputFileDatasetConfig
from azureml.data.dataset_consumption_config import DatasetConsumptionConfig
from azureml.core import Environment


ws = Workspace.from_config()
datastore = ws.get_default_datastore()

#Retrieve an already attached Azure Machine Learning Compute
from azureml.core.compute_target import ComputeTargetException
aml_compute_target = "Your AML Compute"
try:
    aml_compute = AmlCompute(ws, aml_compute_target)
    print("Found existing compute target: {}".format(aml_compute_target))
except ComputeTargetException:
    print("Creating new compute target: {}".format(aml_compute_target))
    
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_D2_V2",
                                                                min_nodes = 1, 
                                                                max_nodes = 1)    
    aml_compute = ComputeTarget.create(ws, aml_compute_target, provisioning_config)
    aml_compute.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

# get datasets from CI
# Ad hoc Datasets also referred to Saved datasets are not discoverable from Audience Insights currently.
# Example: IncompatibleDataset = Dataset.Tabular.from_delimited_files('https://cidatasets.blob.core.windows.net/hotel-scenario/ContosoHotel_HotelActivity_1.csv')
# You need to use registered datasets and then consume them by their name 
# Example: CompatibleDataset = Dataset.get_by_name(ws, name='Hotel Stay Activity Data')

#HotelStayActivity_dataset = Dataset.Tabular.from_delimited_files('https://cidatasets.blob.core.windows.net/hotel-scenario/ContosoHotel_HotelActivity_1.csv')
HotelStayActivity_dataset = Dataset.get_by_name(ws, name='Hotel Stay Activity Data')
HotelStayActivity_pipeline_param = PipelineParameter(name="HotelStayActivity_pipeline_param", default_value=HotelStayActivity_dataset)
HotelStayActivity_ds_consumption = DatasetConsumptionConfig("HotelStayActivity_dataset", HotelStayActivity_pipeline_param)

#Customer_dataset = Dataset.Tabular.from_delimited_files('https://cidatasets.blob.core.windows.net/hotel-scenario/Customer_1.csv')    
Customer_dataset = Dataset.get_by_name(ws, name= 'Customer Data')
Customer_pipeline_param = PipelineParameter(name="Custolmer_pipeline_param", default_value=Customer_dataset)
Customer_ds_consumption = DatasetConsumptionConfig("Customer_dataset", Customer_pipeline_param)

#ServiceUsage_dataset = Dataset.Tabular.from_delimited_files('https://cidatasets.blob.core.windows.net/hotel-scenario/ContosoHotel_ServiceUsage_1.csv')    
ServiceUsage_dataset = Dataset.get_by_name(ws, name='Service Usage Data')
ServiceUsage_pipeline_param = PipelineParameter(name="ServiceUsage_pipeline_param", default_value=ServiceUsage_dataset)
ServiceUsage_ds_consumption = DatasetConsumptionConfig("ServiceUsage_dataset", ServiceUsage_pipeline_param)

# Currently OutputFIleDatasetConfig doesn't support parameterization of the output datastore and path which are requried for Audience Insights Integration
# Example: hotel_churn_ds = OutputFileDatasetConfig(name="HotelChurnOutput", destination=(datastore,"path")).read_delimited_files().register_on_complete(name='HotelChurnOutput')
# Instead, you you will need to use string parameters for datastore and path, and then rely on Datastore.upload() to upload it to the datastore and path, which is accessible to Audience Insights.
# In your Pipeline Setup: OutputPathParameter = PipelineParameter(name="output_path", default_value="HotelChurnOutput1/HotelChurnOutput.csv")
# In your score.py: ws = run.experiment.workspace
#                   datastore = Datastore.get(ws, datastore_parameter)
#                   datastore.upload(src_dir=compute_src_dir, target_path=output_path, overWrite=True)

OutputPathParameter = PipelineParameter(name="output_path", default_value="HotelChurnOutput2/HotelChurnOutput.csv")
OutputDatastoreParameter = PipelineParameter(name="output_datastore", default_value="workspaceblobstore")

env = Environment.from_conda_specification(name = 'env', file_path = './HotelChurnInferencePipeline.yml')

# source_directory
source_directory = './scripts'
os.makedirs(source_directory, exist_ok=True)
script_name = "score.py"
# define a single step pipeline for demonstration purpose.
src = ScriptRunConfig(source_directory=source_directory,
                    script=script_name,
                    compute_target=aml_compute_target,
                    environment=env)

inferenceStep = PythonScriptStep(
    name="Inferencing_Step",
    script_name=src.script,
    arguments=["--input_data1", HotelStayActivity_ds_consumption, "--input_data2", Customer_ds_consumption, "--input_data3", ServiceUsage_ds_consumption, "--output_path", OutputPathParameter, "--output_datastore", OutputDatastoreParameter],
    inputs=[HotelStayActivity_ds_consumption, Customer_ds_consumption, ServiceUsage_ds_consumption], 
    #compute_target=aml_compute_target, 
    source_directory=src.source_directory,
    runconfig = src.run_config)

print("Inferencing_Step created")
# build and validate Pipeline
pipeline = Pipeline(workspace=ws, steps=[inferenceStep])
print("Pipeline is built")

In [1]:
#register model

from azureml.core.model import Model
# Tip: When model_path is set to a directory, you can use the child_paths parameter to include
#      only some of the files from the directory
model = Model.register(model_path = "./models",
                       model_name = "churnscore",
                       description = "Logistic Regression Model",
                       workspace = ws)

In [1]:
# publish pipeline endpoint
from azureml.pipeline.core import PipelineEndpoint
#pipeline_endpoint = PipelineEndpoint.publish(workspace=ws, name="HotelChurnPipelineEndpoint", pipeline=pipeline, description="Hotel Churn Pipeline Endpoint")
published_pipeline = pipeline.publish(name="HotelChurnPipeline", description="published pipeline")
pipeline_endpoint = PipelineEndpoint.get(workspace=ws, name="HotelChurnPipelineEndpoint") 
pipeline_endpoint.add_default(pipeline=published_pipeline)

In [25]:
from azureml.core import Experiment
pipeline_endpoint = PipelineEndpoint.get(workspace=ws, name="HotelChurnPipelineEndpoint")
experiment = Experiment(ws, 'my-experiments')
pipeline_run = experiment.submit(config=pipeline_endpoint)
pipeline_run.get_details()